In [66]:
import os
import re
import json
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import random

Below cell is temporarily commented - its execution is not important.

In [67]:
"""
import pickle
archivo_pickle = 'SPAM.pickle'
with open(archivo_pickle, 'rb') as file:
    # Carga los datos del archivo pickle
    spam_model = pickle.load(file)
"""

"\nimport pickle\narchivo_pickle = 'SPAM.pickle'\nwith open(archivo_pickle, 'rb') as file:\n    # Carga los datos del archivo pickle\n    spam_model = pickle.load(file)\n"

In [68]:
class email:
    def __init__(self, id, message_id, data, subject, sender, receiver, day_week, time, time_zone, date, is_reply, length, relative_path, body): #beginning, farewell,
        self._id = id
        self._message_id = message_id
        self._data = data
        
        # Verificar si 'subject' es una lista y convertirlo a cadena si es necesario
        if isinstance(subject, list):
            subject = " ".join(subject)
        
        self._subject = subject
        self._sender = sender
        self._receiver = receiver
        self._number_of_receivers = len(receiver)
        self._day_week = day_week
        self._time = time
        self._time_zone = time_zone
        self._date = date
        self._responded = False 
        self._responses = set()
        #self._beginning = beginning
        #self._farewell = farewell
        self._length = length

        # Determinar si el correo es una respuesta y almacenar el resultado
        self._is_reply = is_reply
        
        self._relative_path = relative_path
        self._body = body

# Luego, al crear una instancia de 'email', esta lógica se ejecutará automáticamente


In [69]:
def email_to_dict(email):
    return email.__dict__

In [70]:
def read_email(filename, message_ids, id):
    with open(filename, 'r', errors='ignore') as file:
        data = file.read()
        id = id
        Date = r'Date: (\w+), (\d+ \w+ \d+) (\d+:\d+:\d+) (-\d{4} \(.*?\))'
        coincidencias = re.findall(Date, data)
        for day_week, date, time, time_zone in coincidencias:
            day_week = day_week
            date = date
            time = time
            time_zone = time_zone
            
        subject = re.findall(r'Subject: (.*)',data)
        
        message_id = re.search(r'Message-ID: (.*)',data)
        if message_id:
            message_id = message_id.group(1)
            
        sender_match = re.search(r'\nFrom: (.*)', data)
        if sender_match:
            sender = sender_match.group(1)
        else:
            sender = None
        
        receiver = re.findall(r'\nTo: (.*)',data)
        input_data = str(receiver)[1:-1]
        receiver_list = input_data.split(', ')
        receiver_list = [receiver.strip() for receiver in receiver_list]
        receiver_list = [s.replace("'", "") for s in receiver_list]
        receiver = [s.replace("\"", "'") for s in receiver_list]
        
        
        #beginning_pattern = (r"(?:\b(?:hello|hi|hey|good (?:morning|afternoon|evening)|greetings|dear|my dearest|hi there|hello there|hi all|hi everyone|hello everyone|dearest|yo|what's up|hey pal|yolo|howdy|how's it going|how are you|what's happening|what's new|good to see you|long time no see|it's been a while|what's cracking|how's everything))\b")
        #beginning = re.findall(beginning_pattern, data, flags=re.IGNORECASE)
        
        
        #farewell_pattern = r"(?:\b(?:Sincerely|Regards|Best regards|Kind regards|Yours sincerely|Yours faithfully|Best wishes|Take care|Warm regards|With gratitude|With best regards|Cheers|Cordially|Respectfully|Until next time|All the best|Have a great day|Talk to you soon|Looking forward to hearing from you|With thanks|Later|Catch you later|Farewell|Peace out|Stay well|All my best|Goodbye|ByeFond regards|Take it easy|So long|See you around|Keep in touch|With warm regards|With all my heart)\b)"
        #farewell = re.findall(farewell_pattern, data, flags=re.IGNORECASE)
        
        if re.search(r"-----Original Message-----", data):
            is_reply = 1
        elif subject[0].startswith("Re:"):
            is_reply = 1
        else:
            is_reply = 0
        
        
        body_b = data.find("X-FileName:")
        body_e = data.find("-----Original Message-----")

        if body_b != -1:
            end_l = data.find("\n", body_b)
        if end_l != -1:
            if body_e != -1:
                body = data[end_l + 1:body_e]  # Extrae el cuerpo hasta "-----Original Message-----"
            else:
                body = data[end_l + 1:]  # Si no hay "-----Original Message-----", toma todo hasta el final

        words = body.split()
        length = len(words)

        relative_path = os.path.relpath(filename, start=os.path.join(os.getcwd(), '..'))
        
       
        #vectorizer = TfidfVectorizer(stop_words='english')
        #data_pre= vectorizer.fit_transform(data)

    
    
        if message_id in message_ids:
            return None
        else:
            email_object = email(id, message_id, data, subject, sender, receiver, day_week,time, time_zone, date, is_reply, length, relative_path, body) #beginning,farewell,
            return email_object

In [71]:
list_of_emails = []
set_of_message_ids = set()
counter = 0


# base_dir = "../maildir"
#base_dir = os.path.join(os.path.dirname(os.getcwd()), 'CODE\maildir')
dir = 'DATA\enron_mail_20150507\maildir2' 
# Here paste your local path to the mails directory
dir = '/home/makowskitomasz/enron/all_mails_dir'

for root, dirs, files in os.walk(dir): #
    for file in files:
        file_path = os.path.join(root, file)
        tmp_email = read_email(file_path, set_of_message_ids, counter)
        counter += 1
        if tmp_email is not None:
            list_of_emails.append(tmp_email)
            set_of_message_ids.add(tmp_email._message_id)

The cell below could be uncommented if we want to obtain smaller sample of emails for faster computations.

In [72]:
"""
random.seed(42)
list_of_emails = random.sample(list_of_emails, 24000)
"""

'\nrandom.seed(42)\nlist_of_emails = random.sample(list_of_emails, 24000)\n'

In [73]:
print(len(list_of_emails))
print(len(set_of_message_ids))

248113
248113


In [74]:
count_replies = 0
for email in list_of_emails:
    if email._is_reply == 1:
        count_replies += 1

print(f'Total number of mails that are responses: {count_replies}')

Total number of mails that are responses: 95910


In [75]:
def find_parent_mail(email, list_of_emails):
    subject = re.sub(r'Re: ', '', email._subject)
    subject = subject.replace(' ', '')
    
    email_datetime_str = email._date + ' ' + email._time
    email_datetime = datetime.strptime(email_datetime_str, '%d %b %Y %H:%M:%S')
    
    for parent_email in list_of_emails:
        if parent_email._id == email._id:
            continue

        parent_subject = re.sub(r'Re: ', '', parent_email._subject)
        parent_subject = parent_subject.replace(' ', '')       
       
        
        if parent_subject == subject:
            if parent_email._sender in email._receiver:
                parent_datetime_str = parent_email._date + ' ' + parent_email._time
                parent_datetime = datetime.strptime(parent_datetime_str, '%d %b %Y %H:%M:%S')
                if parent_datetime < email_datetime:
                    parent_email._responded = True
                    parent_email._responses.add(email._id) 
                    break

In [77]:
for email in list_of_emails:
    if email._is_reply == 1:
        find_parent_mail(email, list_of_emails)

In [78]:
for email in list_of_emails:
    email._responses = list(email._responses)

In [12]:
# for counter in range(1, len(list_of_emails), 100000):
#     emails_dict_list = [email_to_dict(email) for email in list_of_emails[(counter - 1):(counter + 99999)]]
#     file_path = f"C:/3CD/2o cuatri/PROY III/DATA{(counter // 100000) + 1}.json"
#     with open(file_path, 'w') as json_file:
#         json.dump(emails_dict_list, json_file)

We create the df with the columns we want

In [85]:
# Suponemos que list_of_emails es tu lista de objetos 'email'
# Crear un DataFrame directamente desde los atributos de los objetos 'email'
data = {
    'id': [email._id for email in list_of_emails],
    'message_id': [email._message_id for email in list_of_emails],
    'subject': [email._subject for email in list_of_emails],
    'sender': [email._sender for email in list_of_emails],
    'receiver': [email._receiver for email in list_of_emails],
    'day_week': [email._day_week for email in list_of_emails],
    'time': [email._time for email in list_of_emails],
    'time_zone': [email._time_zone for email in list_of_emails],
    'date': [email._date for email in list_of_emails],
    #'beginning': [email._beginning for email in list_of_emails],
    #'farewell': [email._farewell for email in list_of_emails],
    'is_reply': [email._is_reply for email in list_of_emails],
    'length': [email._length for email in list_of_emails],
    # 'is_reply': [email._is_reply for email in list_of_emails],
    'responses': [list(email._responses) for email in list_of_emails],  # Asumiendo que _responses es un conjunto
    #'data':[email._data for email in list_of_emails],
    'body': [email._body for email in list_of_emails],
    'number_of_receivers':[ email._number_of_receivers for mail in list_of_emails]
}
pd.set_option('display.max_colwidth', 50)
emails_df = pd.DataFrame(data)

In [86]:
emails_df.head(5)

,id,message_id,subject,sender,receiver,day_week,time,time_zone,date,is_reply,length,responses,body,number_of_receivers
0,0,<32775907.1075849805605.JavaMail.evans@thyme>,RE: PPA - Notice of Intent to Terminate RE: PP...,rob.gay@enron.com,"[rkg@tozzini.com.br, <Richard.A.Lammers@enron....",Wed,09:15:00,-0800 (PST),20 Dec 2000,0,2283,[],\nWhat is the date of the law referening the F...,1
1,1,<14227041.1075843704275.JavaMail.evans@thyme>,Re: FERC ruling on Kern expansion Re: FERC rul...,stephanie.miller@enron.com,"[jeff.dasovich@enron.com, Stephanie Miller/Cor...",Tue,01:55:00,-0700 (PDT),10 Apr 2001,1,888,[],\n---------------------- Forwarded by Stephani...,1
2,2,<9931092.1075841720154.JavaMail.evans@thyme>,There's Changes in the FERC,enron.announcements@enron.com,[all.worldwide@enron.com],Mon,18:08:00,-0700 (PDT),16 Apr 2001,0,54,[],\nRead about the two new FERC Commissioners.\n...,1
3,3,<5963082.1075855955350.JavaMail.evans@thyme>,Re: January 11 - Managing Operational Risk Jan...,sally.beck@enron.com,"[jefferson.sorenson@enron.com, Sally Beck/HOU/...",Fri,10:19:00,-0800 (PST),5 Jan 2001,1,463,[],\nI am so sorry to hear about your dad. I did...,1
4,4,<6816183.1075852417428.JavaMail.evans@thyme>,EnronOnline Website Audit,karen.o'day@enron.com,[john.griffith@enron.com],Fri,11:11:04,-0700 (PDT),3 Aug 2001,0,87,[],"\nJohn,\n\nWe are working on an EnronOnline We...",1


### Some data preprocessing

In [93]:
# We adjust the time to the sime time_zone

def adjust_to_pst(time_str, tz_str):
    # Convert string to datetime
    time = pd.to_datetime(time_str).time()
    # Create a full datetime object with a fixed date
    datetime_obj = pd.Timestamp(f'2024-04-22 {time.hour}:{time.minute}:{time.second}')
    # Calculate offset from PST in hours
    offset = int(tz_str[:3]) + 8
    # Adjust the time
    adjusted_time = (datetime_obj - pd.Timedelta(hours=offset)).time()
    return adjusted_time
# Apply the function to adjust time to PST
emails_df['adjusted_time'] = emails_df.apply(lambda row: adjust_to_pst(row['time'], row['time_zone']), axis=1)
emails_df[['time', 'time_zone', 'adjusted_time']]


#Make 'length a discrete column
emails_df['length_num'] = emails_df['length']

num_emails_per_group = len(emails_df) // 3

emails_df.sort_values(by='length', inplace=True)

# Step 3: Divide the sorted emails into groups based on their length
short_emails = emails_df.iloc[:num_emails_per_group]
medium_emails = emails_df.iloc[num_emails_per_group:2*num_emails_per_group]
large_emails = emails_df.iloc[2*num_emails_per_group:]

# Step 4: Assign categorical labels to each group
short_emails['length'] = 'short'
medium_emails['length'] = 'medium'
large_emails['length'] = 'large'

# Step 5: Combine the emails from all groups into a single DataFrame
emails_df = pd.concat([short_emails, medium_emails, large_emails])

# Optional: Shuffle the DataFrame to mix the emails from different categories
emails_df = emails_df.sample(frac=1).reset_index(drop=True)

emails_df

/tmp/ipykernel_4706/3937674778.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_emails['length'] = 'short'
/tmp/ipykernel_4706/3937674778.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medium_emails['length'] = 'medium'
/tmp/ipykernel_4706/3937674778.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

,id,message_id,subject,sender,receiver,day_week,time,time_zone,date,is_reply,responses,body,number_of_receivers,adjusted_time,length,length_num
0,84349,<8995059.1075853654310.JavaMail.evans@thyme>,FW: Pictures of Downtown Houston Tunnels Flood...,stacy.dickson@enron.com,[s.john.granmayeh@usa.conoco.com],Mon,06:29:00,-0700 (PDT),11 Jun 2001,0,[],\n----- Forwarded by Stacy E Dickson/HOU/ECT o...,1,05:29:00,medium,46
1,65845,<31353073.1075853894037.JavaMail.evans@thyme>,Capacity for May 17th,mainesa@natfuel.com,"[hormeljw@bp.com, george_caouette@br-inc.ca, b...",Thu,01:58:00,-0700 (PDT),17 May 2001,0,[],\nI'm off to Paris tomorrow until the 29th. P...,1,00:58:00,large,142
2,182755,<17472084.1075845407911.JavaMail.evans@thyme>,Conference Call-Excess LNG Pricing,etelvina.guerra@enron.com,"[rick.sierra@enron.com, jaime.sanabria@enron.c...",Tue,07:22:14,-0700 (PDT),5 Jun 2001,0,[],\n\nThere will be a Conference Call on Thursda...,1,06:22:14,medium,41
3,211551,<27752609.1075862103380.JavaMail.evans@thyme>,Entertainment Order Confirmation,askus@entertainment.com,[e.taylor@enron.com],Fri,09:01:07,-0800 (PST),2 Nov 2001,0,[],"\nDear michael taylor,\n\nThank you for your E...",1,09:01:07,large,303
4,129256,<28793629.1075853115709.JavaMail.evans@thyme>,RE: Children's Festival Tickets Are Here. Com...,zakiyyah.mcclure@enron.com,"[iris.waser@enron.com, lorraine.becker@enron.c...",Thu,13:39:29,-0700 (PDT),18 Oct 2001,1,[],\nDOES ANYONE NEED 2 TICKETS ???? I CANT ATTEN...,1,12:39:29,short,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248108,57812,<12633618.1075853683417.JavaMail.evans@thyme>,Sale to EES for April,chris.germany@enron.com,"[james.barker@enron.com, patricia.downey@enron...",Fri,03:21:00,-0700 (PDT),28 Jul 2000,0,[],"\nDonnie, I prepared this email a couple weeks...",1,02:21:00,large,148
248109,78752,<30932077.1075856597532.JavaMail.evans@thyme>,FERC's Soft Price Caps: What Do They Mean? - C...,webmaster@cera.com,[],Mon,05:00:00,-0700 (PDT),2 Apr 2001,0,[],\n*** PLEASE ACCEPT OUR APOLOGIES IF YOU HAVE ...,1,04:00:00,large,618
248110,121689,<25746772.1075858147905.JavaMail.evans@thyme>,Re: Fw: Instructions for Life,matthew.lenhart@enron.com,[stadogb@email.msn.com],Thu,03:19:00,-0700 (PDT),17 Aug 2000,1,[],\ndon't deny it.,1,02:19:00,short,3
248111,215014,<30919529.1075845186028.JavaMail.evans@thyme>,"$60 rt airfares coast-to-coast thru oct. 26, '01",deals@bestfares.com,[pkeavey@ect.enron.com],Wed,20:24:49,-0700 (PDT),30 May 2001,0,[],\n Today's Hottest Travel Deals from Bestfares...,1,19:24:49,large,1707


In [96]:
emails_df['answered'] = emails_df['responses'].apply(lambda x: 1 if len(x) > 0 else 0)

We save the final df on a json to use for the static model

In [101]:
emails_df.sort_values(by='id', inplace=True)

In [102]:
emails_df.to_json('emails.json', orient='records', lines=True)

Code below is written to store data in the proper JSON format. However, the one above is correct if you use read_json method with corret parameters (orient='records', lines=True)

In [27]:
# with open('emails.json', 'w') as f:
#     f.write("[\n")
    
#     for idx, row in emails_df.iterrows():
#         row_json = row.to_json()
        
#         f.write(row_json)
#         f.write(",\n" if idx < len(emails_df) - 1 else "\n")
    
#     f.write("]\n")
